# Analyzing Hubway Trips in SQL

In [1]:
#You will have to install ipython-sql using the following command:

!pip install jupysql

#Just Execute the cell and it will install


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# If your file is in another Directory (Without Space in Name of Directory) : %sql sqlite:///Hello/Another_Folder/hubway_small_ok.db

# If your file is in Same Directory : %sql sqlite:///hubway_small_ok.db

%reload_ext sql
%sql sqlite:///hubway.db
%config SqlMagic.autocommit=False

Connecting to 'sqlite:///hubway.db'

**Please write your queries below in the empty boxes same as you write in a SQL Window**

Just append **%sql** before your query like this '%sql Select * from Trips limit 5;'

In [3]:
%%sql

Select * from Trips limit 2

Running query in 'sqlite:///hubway.db'

id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male
2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966.0,Male


**About the Dataset and Schema**

The database has two tables, trips and stations. To begin with, we'll look at the trips table. It contains the following columns:


**Trips:**

id — A unique integer that serves as a reference for each trip

duration — The duration of the trip, measured in seconds

start_date — The date and time the trip began

start_station — An integer that corresponds to the id column in the stations table for the station the trip started at

end_date — The date and time the trip ended

end_station — The 'id' of the station the trip ended at

bike_number — Hubway's unique identifier for the bike used on the trip

sub_type — The subscription type of the user. "Registered" for users with a membership, "Casual" for users without a membership

zip_code — The zip code of the user (only available for registered members)

birth_date — The birth year of the user (only available for registered members)

gender — The gender of the user (only available for registered members)


**Station:**


id — A unique identifier for each station (corresponds to the start_station and end_station columns in the trips table)

station — The station name

municipality — The municipality that the station is in (Boston, Brookline, Cambridge or Somerville)

lat — The latitude of the station

lng — The longitude of the station

## Analytical Questions Test in SQL

Q1. What was the duration of the longest trip? What was the average trip duration? What was the smallest trip duration?

In [4]:

%%sql

select max(duration) longest_duration, round(avg(duration),2) avg_duration,min(duration)
Smallest_duration
from trips


Running query in 'sqlite:///hubway.db'

longest_duration,avg_duration,Smallest_duration
9999,912.41,0


Q2. How many trips were taken by 'Registered' users?

Hint: Use Sub Type to Filter by Where

In [5]:
%%sql
SELECT COUNT(*) as registered_trips
FROM trips
WHERE sub_type = 'Registered'

Running query in 'sqlite:///hubway.db'

registered_trips
1105192


Q3. How many trips were taken by male users in comparison to Female users? for Registered users only

Hint: Group By Gender and Calculate Count

In [6]:
%%sql
SELECT gender,
       COUNT(*) as number_of_trips
FROM trips
WHERE sub_type = 'Registered'
  AND gender IS NOT NULL
GROUP BY gender
ORDER BY number_of_trips DESC

Running query in 'sqlite:///hubway.db'

gender,number_of_trips
Male,833858
Female,271333
Female,1


There's an unusual duplicate "Female" row with just 1 trip
There's also an empty row (likely representing NULL or empty values)
To clean this up, we can modify the query to:

In [7]:
%%sql
SELECT gender,
       COUNT(*) as number_of_trips
FROM trips
WHERE sub_type = 'Registered'
  AND gender IN ('Male', 'Female')
GROUP BY gender
ORDER BY number_of_trips DESC

Running query in 'sqlite:///hubway.db'

gender,number_of_trips
Male,833858
Female,271333


Q4. Do registered or casual users take longer trips?

Hint: Group By Sub Type and Calculate Average

In [8]:
%%sql
SELECT sub_type,
       COUNT(*) as number_of_trips,
       ROUND(AVG(duration), 2) as avg_duration_seconds
FROM trips
GROUP BY sub_type
ORDER BY avg_duration_seconds DESC

Running query in 'sqlite:///hubway.db'

sub_type,number_of_trips,avg_duration_seconds
Casual,464809,1519.64
Registered,1105192,657.03


converting same into minutes

In [9]:
%%sql
SELECT sub_type,
       COUNT(*) as number_of_trips,
       ROUND(AVG(duration)/60.0, 2) as avg_duration_minutes  -- Converting seconds to minutes
FROM trips
GROUP BY sub_type
ORDER BY avg_duration_minutes DESC

Running query in 'sqlite:///hubway.db'

sub_type,number_of_trips,avg_duration_minutes
Casual,464809,25.33
Registered,1105192,10.95


Q5. Which bike was used for the most trips?

Hint : Use Bike Number and Limit Results to 1

In [10]:
%%sql
SELECT bike_number,
       COUNT(*) as number_of_trips
FROM trips
GROUP BY bike_number
ORDER BY number_of_trips DESC
LIMIT 1

Running query in 'sqlite:///hubway.db'

bike_number,number_of_trips
B00490,2120


Q6. What is the average duration of trips by users over the age of 30?

Hint : DoB is already in years and use Airthmetic operator to caluculate Age like (2025-DOB)

Link : #GETTING AGE IN SQLLITE : https://stackoverflow.com/questions/49929953/how-to-calculate-approximate-age-based-on-birth-year

In [12]:
%%sql
SELECT ROUND(AVG(duration), 2) as avg_duration_seconds
FROM trips
WHERE sub_type = 'Registered'
  AND birth_date IS NOT NULL
  AND (2015 - birth_date) > 30

Running query in 'sqlite:///hubway.db'

avg_duration_seconds
661.5


Q7. Which stations are most frequently used for round trips?

Hint: Round trip is considered as where Start and Stop Stations are same. Limit to top 5 Stations

In [13]:
%%sql
SELECT s.station as station_name,
       COUNT(*) as round_trips
FROM trips t
JOIN stations s ON t.start_station = s.id
WHERE t.start_station = t.end_station
GROUP BY t.start_station
ORDER BY round_trips DESC
LIMIT 5

Running query in 'sqlite:///hubway.db'

station_name,round_trips
The Esplanade - Beacon St. at Arlington St.,3064
Boston Public Library - 700 Boylston St.,2548
Boylston St. at Arlington St.,2163
Beacon St / Mass Ave,2144
Newbury St / Hereford St,1636


Q8. How many trips start and end in different municipalities?

Hint: We need to JOIN the trips table to the stations table twice. Once ON the start_station column and then ON the end_station

In [14]:
%%sql
SELECT COUNT(*) as different_municipality_trips
FROM trips t
JOIN stations s1 ON t.start_station = s1.id
JOIN stations s2 ON t.end_station = s2.id
WHERE s1.municipality != s2.municipality

Running query in 'sqlite:///hubway.db'

different_municipality_trips
309748


Q9. How many trips incurred additional fees (lasted longer than 30 minutes)?

Hint: Use an Arithmetic function to calculate Duration in Minutes and Use in Where Clause

In [3]:
%%sql
SELECT COUNT(*) as trips_with_fees
FROM trips
WHERE (duration/60 > 30)

Running query in 'sqlite:///hubway.db'

trips_with_fees
119917


Q10. Which bike was used for the longest total time? Provide the answer in Hours

Hint: Use an Group by Bike Number to sum Duration in Seconds and Limit Result to 1

In [21]:
%%sql
SELECT bike_number,
       ROUND(SUM(duration)/3600.0, 2) as total_hours
FROM trips
GROUP BY bike_number
ORDER BY total_hours DESC
LIMIT 1

Running query in 'sqlite:///hubway.db'

bike_number,total_hours
B00490,571.85


Q11. Did registered or casual users take more round trips?

Hint: Recall Q7 and Modify your query to answer by Sub Type

In [22]:
%%sql
SELECT sub_type,
       COUNT(*) as round_trips
FROM trips
WHERE start_station = end_station  -- Round trips have same start and end station
GROUP BY sub_type
ORDER BY round_trips DESC

Running query in 'sqlite:///hubway.db'

sub_type,round_trips
Casual,41427
Registered,31635


Q12. Which municipality had the most frequent Station End?

Hint: Join the Station table on End Station and then Group by Municipality to find answer

In [23]:
%%sql
SELECT s.municipality,
       COUNT(*) as end_station_count
FROM trips t
JOIN stations s ON t.end_station = s.id 
GROUP BY s.municipality
ORDER BY end_station_count DESC
LIMIT 1

Running query in 'sqlite:///hubway.db'

municipality,end_station_count
Boston,1212364


Q13. Which From and To Routes are Frequent with Regards to Municipalities?

Hint: Get Start and End Municipalities and Group by Both Attributes for Count of Rides

In [24]:
%%sql
SELECT 
    s1.municipality as start_municipality,
    s2.municipality as end_municipality,
    COUNT(*) as number_of_trips
FROM trips t
JOIN stations s1 ON t.start_station = s1.id
JOIN stations s2 ON t.end_station = s2.id
GROUP BY s1.municipality, s2.municipality
ORDER BY number_of_trips DESC

Running query in 'sqlite:///hubway.db'

start_municipality,end_municipality,number_of_trips
Boston,Boston,1081805
Cambridge,Cambridge,162538
Boston,Cambridge,110968
Cambridge,Boston,110078
Cambridge,Somerville,20998
Somerville,Cambridge,20765
Boston,Brookline,14838
Brookline,Boston,14573
Somerville,Somerville,12746
Boston,Somerville,6071


Q14. Which Age Band in "Registered Users" has the most Rides? Calculate Age Bands based on interval of 10 and Cut Off by 70 or Above

Hint: Use Case When to Create Range of Value Bands as per Above Criteria then Group By. Use DoB Years to Calculate Age First (https://youtu.be/G46GIIRbO-g?t=76)

In [25]:
%%sql
SELECT 
    CASE 
        WHEN (2015 - birth_date) >= 70 THEN '70 or Above'
        WHEN (2015 - birth_date) >= 60 THEN '60-69'
        WHEN (2015 - birth_date) >= 50 THEN '50-59'
        WHEN (2015 - birth_date) >= 40 THEN '40-49'
        WHEN (2015 - birth_date) >= 30 THEN '30-39'
        WHEN (2015 - birth_date) >= 20 THEN '20-29'
        ELSE 'Under 20'
    END as age_band,
    COUNT(*) as number_of_rides
FROM trips
WHERE sub_type = 'Registered'
    AND birth_date IS NOT NULL
GROUP BY age_band
ORDER BY number_of_rides DESC

Running query in 'sqlite:///hubway.db'

age_band,number_of_rides
30-39,137171
20-29,77557
40-49,68692
50-59,47590
60-69,17801
70 or Above,1205


# Analyzing Hubway Trips in Python

In [1]:
# MANDATORY CELL TO EXECUTE FIRST SO WE HAVE DATA IN THE RIGHT TYPE

import sqlite3

import pandas as pd, numpy as np

cnx = sqlite3.connect('hubway.db')

cnx.cursor().executescript("""

UPDATE trips SET duration       = NULL WHERE duration IS '';
UPDATE trips SET start_date     = NULL WHERE start_date IS '';
UPDATE trips SET start_station  = NULL WHERE start_station IS '';
UPDATE trips SET end_date       = NULL WHERE end_date IS '';
UPDATE trips SET end_station    = NULL WHERE end_station IS '';
UPDATE trips SET sub_type       = NULL WHERE sub_type IS '';
UPDATE trips SET zip_code       = NULL WHERE zip_code IS '';
UPDATE trips SET birth_date     = NULL WHERE birth_date IS '';
UPDATE trips SET gender         = NULL WHERE gender IS '';

""")

trips=pd.read_sql_query("SELECT * FROM trips", cnx)
stations=pd.read_sql_query("SELECT * FROM stations", cnx)

print(len(trips))
print(len(stations))

trips=trips\
    .replace(r'^\s*$', np.nan, regex=True)\
    .astype({\
            'start_date':'datetime64[ns]'\
            ,'end_date':'datetime64[ns]'\
            ,'start_station':pd.Int64Dtype()\
            ,'end_station':pd.Int64Dtype()\
            ,'birth_date':pd.Int64Dtype()\
            })

stations=stations\
    .replace(r'^\s*$', np.nan, regex=True)\
    .astype({\
            'lat':'float64'\
            ,'lng':'float64'\
            })

print(len(trips))
print(len(stations))

cnx.close()

1570001
142
1570001
142


In [60]:
# ONLY RUN IF YOU HAVE ERROR IN ABOVE CELL (LIKE CANNOT FIND SQLLITE3)

#You will have to install SQLLite Connector using the following command:

!pip install pysqlite3

#Just Execute the cell and it will install

#THEN USE import pysqlite3 as sqlite3 in the import as replacement

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pysqlite3


  error: subprocess-exited-with-error
  
  × Building wheel for pysqlite3 (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [10 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-313\pysqlite3
      copying pysqlite3\dbapi2.py -> build\lib.win-amd64-cpython-313\pysqlite3
      copying pysqlite3\__init__.py -> build\lib.win-amd64-cpython-313\pysqlite3
      running build_ext
      Builds a C extension linking against libsqlite3 library
      building 'pysqlite3._sqlite3' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pysqlite3

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m 

In [2]:
# CHECKING IF THE DATA IS LOAD INTO RIGHT TABLES IN PYTHON DF

print(trips.head(5))

print('\n ------- \n')

print(stations.head(5))

   id  duration          start_date  start_station            end_date  \
0   1         9 2011-07-28 10:12:00             23 2011-07-28 10:12:00   
1   2       220 2011-07-28 10:21:00             23 2011-07-28 10:25:00   
2   3        56 2011-07-28 10:33:00             23 2011-07-28 10:34:00   
3   4        64 2011-07-28 10:35:00             23 2011-07-28 10:36:00   
4   5        12 2011-07-28 10:37:00             23 2011-07-28 10:37:00   

   end_station bike_number    sub_type zip_code  birth_date  gender  
0           23      B00468  Registered   '97217        1976    Male  
1           23      B00554  Registered   '02215        1966    Male  
2           23      B00456  Registered   '02108        1943    Male  
3           23      B00554  Registered   '02116        1981  Female  
4           23      B00554  Registered   '97214        1983  Female  

 ------- 

   id                             station municipality        lat        lng
0   3              Colleges of the Fenway     

In [3]:
# EXAMPLE SOLUTION, REMEMBER YOUR ANSWER SHOULD BE PRECISE AND ON POINT (SORTED AND IN RIGHT FORMAT)

##PYTHON SOLUTION

trips.agg({'duration': ['mean', 'min', 'max']}).T

,mean,min,max
duration,912.409682,0.0,9999.0


In [4]:
## SOLUTION TO QUESTION 2

# Alternative method using value_counts()
registered_count = trips['sub_type'].value_counts()['Registered']
print(f"Number of trips by registered users: {registered_count:,}")

Number of trips by registered users: 1,105,192


In [5]:
## SOLUTION TO QUESTION 3

# Filter for registered users and count trips by gender
gender_counts = trips[
    (trips['sub_type'] == 'Registered') &  # Only registered users
    (trips['gender'].isin(['Male', 'Female']))  # Only Male/Female values
]['gender'].value_counts()

print("Number of trips by gender (registered users only):")
print(gender_counts)


Number of trips by gender (registered users only):
gender
Male      833858
Female    271333
Name: count, dtype: int64


In [6]:
## SOLUTION TO QUESTION 4
# Group by sub_type and calculate count and average duration
trip_analysis = trips.groupby('sub_type').agg({
    'duration': ['count', 'mean']
}).round(2)

# Rename columns for clarity
trip_analysis.columns = ['number_of_trips', 'avg_duration_seconds']

print("Trip Analysis by User Type:")
print(trip_analysis)


Trip Analysis by User Type:
            number_of_trips  avg_duration_seconds
sub_type                                         
Casual               464809               1519.64
Registered          1105192                657.03


In [7]:
## SOLUTION TO QUESTION 5

print("Top bike usage:")
print(trips['bike_number'].value_counts().nlargest(1))

Top bike usage:
bike_number
B00490    2120
Name: count, dtype: int64


In [8]:
## SOLUTION TO QUESTION 6

# Calculate average duration for users over 30
avg_duration = trips[
    (trips['sub_type'] == 'Registered') &
    (trips['birth_date'].notna()) &
    ((2015 - trips['birth_date']) > 30)
]['duration'].mean()

print(f"Average duration for users over 30: {avg_duration:.2f} seconds")

Average duration for users over 30: 661.50 seconds


In [ ]:
## SOLUTION TO QUESTION 7

round_trips = (
    # Join trips with stations
    trips.merge(
        stations[['id', 'station']],
        left_on='start_station',
        right_on='id',
        how='inner'
    )
    # Filter for round trips
    .query('start_station == end_station')
    # Group by start_station ID (not station name)
    .groupby('start_station')
    .size()
    # Sort and get top 5
    .sort_values(ascending=False)
    .head(5)
    # Get the station names
    .reset_index()
    .merge(stations[['id', 'station']], left_on='start_station', right_on='id')
)

print("\nTop 5 stations for round trips:")
for _, row in round_trips.iterrows():
    print(f"{row['station']}: {row[0]}")


Top 5 stations for round trips:
The Esplanade - Beacon St. at Arlington St.: 3064
Boston Public Library - 700 Boylston St.: 2548
Boylston St. at Arlington St.: 2163
Beacon St / Mass Ave: 2144
Newbury St / Hereford St: 1636


In [ ]:
## SOLUTION TO QUESTION 8

# Count trips between different municipalities
different_municipality_trips = (
    # Join with stations table for start municipality
    trips.merge(
        stations[['id', 'municipality']],
        left_on='start_station',
        right_on='id',
        how='inner'
    )
    # Join with stations table again for end municipality
    .merge(
        stations[['id', 'municipality']],
        left_on='end_station',
        right_on='id',
        how='inner',
        suffixes=('_start', '_end') 
    )
    # Filter for different municipalities
    .query('municipality_start != municipality_end')
    .shape[0]  # Count rows
)

print(f"Number of trips between different municipalities: {different_municipality_trips:,}")

Number of trips between different municipalities: 309,748


In [11]:
## SOLUTION TO QUESTION 9

# Count trips with additional fees using the same logic as SQL
trips_with_fees = trips[trips['duration'] // 60 > 30]  # 30 minutes in minutes
trips = len(trips_with_fees)
print(f"trips_with_fees\n{trips}")

trips_with_fees
119917


In [14]:
## SOLUTION TO QUESTION 10

# Ensure trips is a DataFrame
import pandas as pd
import sqlite3

# Connect to the database
cnx = sqlite3.connect('hubway.db')

# Load the trips data into a DataFrame
trips = pd.read_sql_query("SELECT * FROM trips", cnx)

# Close the connection
cnx.close()

# Calculate total duration for each bike and convert to hours
longest_bike_usage = (
    trips.groupby('bike_number')
    .agg(total_duration=('duration', 'sum'))  # Sum duration for each bike
    .reset_index()
    .sort_values(by='total_duration', ascending=False)
    .head(1)  # Get the bike with the longest total duration
)

# Convert total duration from seconds to hours
longest_bike_usage['total_hours'] = longest_bike_usage['total_duration'] / 3600.0

# Print the result
print(f"Bike used for the longest total time: {longest_bike_usage['bike_number'].values[0]}")
print(f"Total hours: {longest_bike_usage['total_hours'].values[0]:.2f}")

Bike used for the longest total time: B00490
Total hours: 571.85


In [13]:
## SOLUTION TO QUESTION 11

# Count round trips by user type (sub_type)
round_trips_by_type = (
    trips[trips['start_station'] == trips['end_station']]  # Filter for round trips
    .groupby('sub_type')  # Group by user type
    .size()  # Count the number of round trips
)

# Print the results
print("\nRound trips by user type:")
print(round_trips_by_type)


Round trips by user type:
sub_type
Casual        41427
Registered    31635
dtype: int64


In [14]:
## SOLUTION TO QUESTION 12

# Count the number of trips ending in each municipality
end_station_counts = (
    trips.merge(
        stations[['id', 'municipality']],
        left_on='end_station',
        right_on='id',
        how='inner'
    )
    .groupby('municipality')
    .size()  # Count the number of trips for each municipality
    .reset_index(name='end_station_count')  # Reset index and name the count column
    .sort_values(by='end_station_count', ascending=False)  # Sort by count
    .head(1)  # Get the municipality with the most trips
)

# Print the result
print(f"Municipality with the most frequent station end: {end_station_counts['municipality'].values[0]}")
print(f"Number of trips: {end_station_counts['end_station_count'].values[0]}")

Municipality with the most frequent station end: Boston
Number of trips: 1212364


In [15]:
## SOLUTION TO QUESTION 13

# Get start and end municipalities and count the number of trips
frequent_routes = (
    trips.merge(stations[['id', 'municipality']], left_on='start_station', right_on='id', how='inner')
    .merge(stations[['id', 'municipality']], left_on='end_station', right_on='id', how='inner', suffixes=('_start', '_end'))
    .groupby(['municipality_start', 'municipality_end'])
    .size()  # Count the number of trips for each route
    .reset_index(name='number_of_trips')  # Reset index and name the count column
    .sort_values(by='number_of_trips', ascending=False)  # Sort by trip count
)

# Print the top routes
print("\nTop routes by municipality:")
print(frequent_routes.head(10))  # Display the top 10 routes


Top routes by municipality:
   municipality_start municipality_end  number_of_trips
0              Boston           Boston          1081805
10          Cambridge        Cambridge           162538
2              Boston        Cambridge           110968
8           Cambridge           Boston           110078
11          Cambridge       Somerville            20998
14         Somerville        Cambridge            20765
1              Boston        Brookline            14838
4           Brookline           Boston            14573
15         Somerville       Somerville            12746
3              Boston       Somerville             6071


In [16]:
## SOLUTION TO QUESTION 14 (MINI CHALLENGE)

#HELP HINT: https://stackoverflow.com/questions/34165876/trying-to-create-grouped-variable-in-python

# Create age bands based on the birth_date
age_bands = (
    trips[trips['sub_type'] == 'Registered']  # Filter for registered users
    .assign(age=lambda x: 2015 - x['birth_date'])  # Calculate age
    .dropna(subset=['age'])  # Drop rows with NaN ages
    .assign(age_band=lambda x: pd.cut(x['age'], 
                                        bins=[0, 20, 30, 40, 50, 60, 70, 100], 
                                        labels=['Under 20', '20-29', '30-39', '40-49', '50-59', '60-69', '70 or Above'], 
                                        right=False))  # Create age bands
    .groupby('age_band')
    .size()  # Count the number of rides in each age band
    .reset_index(name='number_of_rides')  # Reset index and name the count column
    .sort_values(by='number_of_rides', ascending=False)  # Sort by number of rides
)

# Print the result
print("\nAge band with the most rides:")
print(age_bands.head(5))  # Display the age band with the most rides


Age band with the most rides:
  age_band  number_of_rides
2    30-39           137171
1    20-29            77557
3    40-49            68692
4    50-59            47590
5    60-69            17801


C:\Users\Cyber Land 1\AppData\Local\Temp\ipykernel_2724\3167529850.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('age_band')


### BONUS PYTHON QUESTION (10 Marks)

##### Can you show a Report in Python "What are the Top 5 States with Most Number of Rides" For Registered Users Only

##### Hints :

1. Use the Sampled Dataframe for your working, else it would take a lot of time to convert many rows in Original table.

2. You will need Data Type Conversion and Cleaning of Zip Code (Don't fear errors / Learn from them)

3. Where do you get the States? READ THE CELL BELOW

!pip install zipcodes

print('\n')

import zipcodes

##### Gets All the Information Available against ZipCode

print(zipcodes.matching('77429')[0].keys())

['zip_code', 'zip_code_type', 'active', 'city', 'acceptable_cities', 'unacceptable_cities', 'state', 'county', 'timezone', 'area_codes', 'world_region', 'country', 'lat', 'long']

##### Sample Working how to Fetch a Against a ZipCode

zipcodes.matching('77429')[0].get('state')

READ MORE > https://github.com/seanpianka/Zipcodes


In [17]:
#EXECUTE THIS CELL TO CREATE SAMPLED DATAFRAME

registered_trips_sampled=trips.query("sub_type=='Registered'").groupby('zip_code').apply(lambda x: x.sample(10, replace=True)).reset_index(drop = True)

C:\Users\Cyber Land 1\AppData\Local\Temp\ipykernel_2724\442081643.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  registered_trips_sampled=trips.query("sub_type=='Registered'").groupby('zip_code').apply(lambda x: x.sample(10, replace=True)).reset_index(drop = True)


In [18]:
## YOUR REPORT SOLUTION IN PYTHON

# USE registered_trips_sampled Dataframe and temporarily work on column zip_code by storing in variable 'x'

x=registered_trips_sampled['zip_code']

# CLEAN zip_code COLUMN FROM PUNCTUATIONS


# REPLACE NAN VALUES WITH STRING '0' FOR API TO WORK


# TRY TO USE LAMBDA FUNCTION IN PYTHON TO GET STATE FOR EACH ROW, ONCE YOUR ARE DONE WITH ABOVE STEPS


# SAVE THE OUTPUT INTO A COLUMN NAMED AS registered_trips_sampled['state']

registered_trips_sampled['state']=x

In [17]:
# WE WILL USE THIS DICTIONARY TO MAP STATE NAMES INSTEAD OF ABBREVIATIONS

state_names = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}


In [18]:
# CREATE ANOTHER COLUMN BASED ON STATE DERIVED TO GET STATE NAMES USING ABOVE DICTIONARY USING MAP FUNCTION, SAVE in STATE_NAME
!pip install zipcodes
import zipcodes

# Sample the registered trips for analysis
registered_trips_sampled = trips.query("sub_type == 'Registered'").sample(3000, random_state=1)

# Clean zip_code column from punctuations and replace NaN values with '0'
registered_trips_sampled['zip_code'] = registered_trips_sampled['zip_code'].str.replace(r'\D', '', regex=True).fillna('0')

# Function to get state from zip code
def get_state(zip_code):
    if zip_code:
        match = zipcodes.matching(zip_code)
        if match:
            return match[0].get('state')
    return None

# Apply the function to get state for each row
registered_trips_sampled['state'] = registered_trips_sampled['zip_code'].apply(get_state)

# Count the number of rides per state
state_counts = registered_trips_sampled['state'].value_counts().reset_index()
state_counts.columns = ['state', 'number_of_rides']
state_counts['state'] = state_counts['state'].replace(state_names)
# Get the top 5 states
top_states = state_counts.head(5)

# Print the result
print("\nTop 5 States with Most Number of Rides for Registered Users:")
print(top_states)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Top 5 States with Most Number of Rides for Registered Users:
           state  number_of_rides
0  Massachusetts             2931
1  New Hampshire               14
2   Rhode Island               13
3    Connecticut                5
4     California                4


### WE HOPE YOU ENJOYED LEARNING SQL & PYTHON TOGHETHER